In [1]:

import pandas as pd 
import numpy as np
import plotly 
from matplotlib import pyplot as plt
import os 
from datetime import datetime, timedelta
# pd.set_option("display.max_rows", None)


In [2]:
data_file_path = "C:/Users/govindd1/Downloads/compenstion_data.csv"

In [3]:
df = pd.read_csv(data_file_path)

c:\Users\govindd1\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (22,33,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


##### * Normal EDA to find the nature of Data 

In [4]:
#checking the Fillrate 
null_rate = (df.isnull().sum()/len(df))*100
null_rate = null_rate.drop(null_rate[null_rate == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' : null_rate})
missing_data

,Missing Ratio
PTD Date,99.885995
PPD Non-Scheduled Loss Date,97.677162
First Appeal Date,96.428129
Section 32 Date,94.534755
Controverted Date,90.357545
PPD Scheduled Loss Date,88.207071
OIICS Secondary Source Description,74.620505
OIICS Secondary Source Code,74.620505
OIICS Injury Source Description,74.619772
OIICS Injury Source Code,74.619772


In [5]:
df.drop(['PTD Date', 'PPD Non-Scheduled Loss Date', 'First Appeal Date', 'Section 32 Date', 'Controverted Date', 'PPD Scheduled Loss Date' ], axis=1, inplace=True)


In [6]:
df.columns

Index(['Accident', 'Accident Date', 'Age at Injury',
       'Alternative Dispute Resolution', 'ANCR Date', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'Birth Year',
       'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'Claim Type', 'Closed Count',
       'County of Injury', 'COVID-19 Indicator', 'Current Claim Status',
       'District Name', 'First Hearing Date', 'Gender', 'Hearing Count',
       'Highest Process', 'IME-4 Count', 'Industry Code',
       'Industry Code Description', 'Interval Assembled to ANCR',
       'Medical Fee Region', 'Occupational Disease',
       'OIICS Event Exposure Code', 'OIICS Event Exposure Description',
       'OIICS Injury Source Code', 'OIICS Injury Source Description',
       'OIICS Nature of Injury Code', 'OIICS Nature of Injury Description',
       'OIICS Part Of Body Code', 'OIICS Part Of Body Description',
       'OIICS Secondary Source Code', 'OIICS Secondary 

In [11]:
df['Current Claim Status'].value_counts()

NO FURTHER ACTION              3923702
ADMINISTRATIVELY CLOSED         362245
CASE CANCELLED                   95324
NEWLY ASSEMBLED                  44616
MOTION SET                       22861
HEARING UNSET                    17195
HEARING SET                      14840
RE-OPENED ADMINISTRATIVELY       14506
RETURNED TO EXAMINING             4396
REFEREE - RESERVED DECISION        112
ARGUMENT SET                         1
Name: Current Claim Status, dtype: int64

In [30]:
# removing less relevant columns 

df.drop([
       'COVID-19 Indicator', 'Highest Process', 'IME-4 Count','C-2 Date',
       'C-3 Date', 'Highest Process', 'IME-4 Count', 'Hearing Count'], axis=1, inplace=True)


In [64]:
df['WCIO Part Of Body Description'].value_counts()

LOWER BACK AREA                                          281757
KNEE                                                     225661
FINGER(S)                                                180835
SHOULDER(S)                                              169276
MULTIPLE BODY PARTS (INCLUDING BODY                      161337
HAND                                                     146663
ANKLE                                                    104356
WRIST                                                     86379
FOOT                                                      83327
SOFT TISSUE                                               82716
LOWER ARM                                                 75909
EYE(S)                                                    65675
THUMB                                                     60769
LOWER LEG                                                 55517
MULTIPLE HEAD INJURY                                      54922
UPPER ARM                               

In [11]:
df = df.dropna(how='all')

In [12]:
df['Age at Injury'].mean()

38.75721021254732

In [13]:
df['Age at Injury'].median()

41.0

In [31]:
df['Accident Date'] = pd.to_datetime(df['Accident Date'])

In [32]:
df['accident_year'] = df['Accident Date'].dt.strftime('%Y')
df['accident_year'] = pd.to_numeric(df['accident_year'], errors='coerce').convert_dtypes() 
# df['accident_year'] 

In [33]:
df['accident_month'] = df['Accident Date'].dt.strftime('%m')
df['accident_month'] = pd.to_numeric(df['accident_month'], errors='coerce').convert_dtypes() 
# df['accident_month'] 

In [34]:
value_counts = df['accident_year'].value_counts()
# value_counts

In [18]:
df['Birth Year'].value_counts()

0       550420
1963    110710
1964    108926
1961    107016
1962    106821
         ...  
1900         3
1906         3
2018         2
2014         2
2012         2
Name: Birth Year, Length: 117, dtype: int64

##### * find the lag in days inbetween each processes

In [53]:
df['Accident Date'] = pd.to_datetime(df['Accident Date'])
df['ANCR Date']  = pd.to_datetime(df['ANCR Date'])
# df['Controverted Date']  = pd.to_datetime(df['Controverted Date'])
# df['Section 32 Date']  = pd.to_datetime(df['Section 32 Date'])
# df['Assembly Date']  = pd.to_datetime(df['Assembly Date'])
# df['']  = pd.to_datetime(df[''])


In [55]:
# days inbetween Accident and reporting to insurance company
df['days_btw_accident_and_claim_estd'] =  (df['ANCR Date'] - df['Accident Date']).dt.days
# df['days_btw_accident_and_claim_estd'].value_counts()

In [56]:
df['days_btw_accident_and_claim_estd'].value_counts()

105.0      11577
98.0       11505
112.0      11407
119.0      11322
126.0      11301
           ...  
14237.0        1
6896.0         1
5986.0         1
7073.0         1
8347.0         1
Name: days_btw_accident_and_claim_estd, Length: 8140, dtype: int64

In [60]:
# (df['days_btw_accident_and_claim_estd'] < timedelta(days = 0)).value_counts()

(df['days_btw_accident_and_claim_estd'] <  0).value_counts()

False    4499798
Name: days_btw_accident_and_claim_estd, dtype: int64

In [22]:
df.columns

Index(['Accident', 'Accident Date', 'Age at Injury',
       'Alternative Dispute Resolution', 'ANCR Date', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'Birth Year',
       'Carrier Name', 'Carrier Type', 'Claim Identifier', 'Claim Injury Type',
       'Claim Type', 'Closed Count', 'County of Injury',
       'Current Claim Status', 'District Name', 'First Hearing Date', 'Gender',
       'Industry Code', 'Industry Code Description',
       'Interval Assembled to ANCR', 'Medical Fee Region',
       'Occupational Disease', 'OIICS Event Exposure Code',
       'OIICS Event Exposure Description', 'OIICS Injury Source Code',
       'OIICS Injury Source Description', 'OIICS Nature of Injury Code',
       'OIICS Nature of Injury Description', 'OIICS Part Of Body Code',
       'OIICS Part Of Body Description', 'OIICS Secondary Source Code',
       'OIICS Secondary Source Description', 'WCIO Cause of Injury Code',
       'WCIO Cause of Injury Description', 'WCIO Natur

In [58]:
# df = df[df['days_btw_accident_and_claim_estd'] > timedelta(days=0)]



df.loc[df['days_btw_accident_and_claim_estd'] < 0 , "days_btw_accident_and_claim_estd"] = np.nan

In [59]:
df.shape

(4499798, 45)

In [61]:
df['days_btw_accident_and_claim_estd']


0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
4499793    166.0
4499794      NaN
4499795      NaN
4499796      NaN
4499797      NaN
Name: days_btw_accident_and_claim_estd, Length: 4499798, dtype: float64

In [46]:
# try:
#     df['days_btw_accident_and_claim_estd'] = df['days_btw_accident_and_claim_estd'].dt.days.astype('int16')
# except:
#     print("handled IntCastingNaNError")
    

handled IntCastingNaNError


In [62]:
df['days_btw_accident_and_claim_estd']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
4499793    166.0
4499794      NaN
4499795      NaN
4499796      NaN
4499797      NaN
Name: days_btw_accident_and_claim_estd, Length: 4499798, dtype: float64

In [ ]:
# days inbetween accident occured and reported to insurance company where the worker has lost employment more than a week
# df['days_btw_accident_and_claim_assembled_morethan_one_week'] =  df['Assembly Date'] - df['Accident Date']
# df['days_btw_accident_and_claim_assembled_morethan_one_week'].value_counts()

In [ ]:
# days inbetween claim registeres and claim compensation negotiated 
# df['days_btw_claim_est_and_negotiated'] = df['Section 32 Date'] - df['ANCR Date']
# df['days_btw_claim_est_and_negotiated'].value_counts()

In [63]:
#removing data before the year 2000
df = df[df['accident_year']>=2000]
df.shape

(4392532, 45)

In [ ]:
df['accident_year'].value_counts(sort = 'Ascending')

##### * grouping similar body parts together and creating another column


In [65]:
df['WCIO Part Of Body Description'].value_counts()

LOWER BACK AREA                                          281757
KNEE                                                     225661
FINGER(S)                                                180835
SHOULDER(S)                                              169276
MULTIPLE BODY PARTS (INCLUDING BODY                      161337
HAND                                                     146663
ANKLE                                                    104356
WRIST                                                     86379
FOOT                                                      83327
SOFT TISSUE                                               82716
LOWER ARM                                                 75909
EYE(S)                                                    65675
THUMB                                                     60769
LOWER LEG                                                 55517
MULTIPLE HEAD INJURY                                      54922
UPPER ARM                               

In [66]:
#Arms

df.loc[(df['WCIO Part Of Body Description'] == "FINGER(S)" ), 'injured_body_part'] = "Arms"
df.loc[(df['WCIO Part Of Body Description'] == "SHOULDER(S)" ), 'injured_body_part'] = "Arms"
df.loc[(df['WCIO Part Of Body Description'] == "HAND" ), 'injured_body_part'] = "Arms"
df.loc[(df['WCIO Part Of Body Description'] == "LOWER ARM" ), 'injured_body_part'] = "Arms"
df.loc[(df['WCIO Part Of Body Description'] == "THUMB" ), 'injured_body_part'] = "Arms"
df.loc[(df['WCIO Part Of Body Description'] == "UPPER ARM" ), 'injured_body_part'] = "Arms"
df.loc[(df['WCIO Part Of Body Description'] == "ELBOW" ), 'injured_body_part'] = "Arms"
df.loc[(df['WCIO Part Of Body Description'] == "WRIST (S) & HAND(S)" ), 'injured_body_part'] = "Arms"

#Face
df.loc[(df['WCIO Part Of Body Description'] == "EYE(S)"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "MULTIPLE HEAD INJURY"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "SKULL"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "FACIAL BONES"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "EAR(S)"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "NOSE"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "MOUTH"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "BRAIN"), 'injured_body_part'] = "Face"
df.loc[(df['WCIO Part Of Body Description'] == "TEETH"), 'injured_body_part'] = "Face"

#Legs
df.loc[(df['WCIO Part Of Body Description'] == "KNEE"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "ANKLE"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "WRIST"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "FOOT"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "LOWER LEG"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "UPPER LEG"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "TOES"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "MULTIPLE LOWER EXTREMITIES"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "GREAT TOE"), 'injured_body_part'] = "Legs"
df.loc[(df['WCIO Part Of Body Description'] == "BUTTOCKS"), 'injured_body_part'] = "Legs"


#multiple
df.loc[(df['WCIO Part Of Body Description'] == "MULTIPLE BODY PARTS (INCLUDING BODY"), 'injured_body_part'] = "Multiple"
df.loc[(df['WCIO Part Of Body Description'] == "MULTIPLE UPPER EXTREMITIES"), 'injured_body_part'] = "Multiple"
df.loc[(df['WCIO Part Of Body Description'] == "BODY SYSTEMS AND MULTIPLE BODY SYSTEMS"), 'injured_body_part'] = "Multiple"
df.loc[(df['WCIO Part Of Body Description'] == "WHOLE BODY"), 'injured_body_part'] = "Multiple"


#others
df.loc[(df['WCIO Part Of Body Description'] == "SOFT TISSUE"), 'injured_body_part'] = "Others"
df.loc[(df['WCIO Part Of Body Description'] == "NO PHYSICAL INJURY"), 'injured_body_part'] = "Others"
df.loc[(df['WCIO Part Of Body Description'] == "INSUFFICIENT INFO TO PROPERLY IDENTIFY - UNCLASSIFIED"), 'injured_body_part'] = "Others"
df.loc[(df['WCIO Part Of Body Description'] == "INTERNAL ORGANS"), 'injured_body_part'] = "Others"
df.loc[(df['WCIO Part Of Body Description'] == "ARTIFICIAL APPLIANCE"), 'injured_body_part'] = "Others"


#Respiratory
df.loc[(df['WCIO Part Of Body Description'] == "LUNGS"), 'injured_body_part'] = "Respiratory"
df.loc[(df['WCIO Part Of Body Description'] == "LARYNX"), 'injured_body_part'] = "Respiratory"
df.loc[(df['WCIO Part Of Body Description'] == "SACRUM AND COCCYX"), 'injured_body_part'] = "Respiratory"
df.loc[(df['WCIO Part Of Body Description'] == "TRACHEA"), 'injured_body_part'] = "Respiratory"


#spinal
df.loc[(df['WCIO Part Of Body Description'] == "LOWER BACK AREA"), 'injured_body_part'] = "Spinal"
df.loc[(df['WCIO Part Of Body Description'] == "UPPER BACK AREA"), 'injured_body_part'] = "Spinal"
df.loc[(df['WCIO Part Of Body Description'] == "MULTIPLE NECK INJURY"), 'injured_body_part'] = "Spinal"
df.loc[(df['WCIO Part Of Body Description'] == "DISC"), 'injured_body_part'] = "Spinal"
df.loc[(df['WCIO Part Of Body Description'] == "LUMBAR & OR SACRAL VERTEBRAE (VERTEBRA"), 'injured_body_part'] = "Spinal"
df.loc[(df['WCIO Part Of Body Description'] == "VERTEBRAE"), 'injured_body_part'] = "Spinal"
df.loc[(df['WCIO Part Of Body Description'] == "SPINAL CORD"), 'injured_body_part'] = "Spinal"


#Torso
df.loc[(df['WCIO Part Of Body Description'] == "CHEST"), 'injured_body_part'] = "Torso"
df.loc[(df['WCIO Part Of Body Description'] == "ABDOMEN INCLUDING GROIN"), 'injured_body_part'] = "Torso"
df.loc[(df['WCIO Part Of Body Description'] == "HIP"), 'injured_body_part'] = "Torso"
df.loc[(df['WCIO Part Of Body Description'] == "MULTIPLE TRUNK"), 'injured_body_part'] = "Torso"
df.loc[(df['WCIO Part Of Body Description'] == "PELVIS"), 'injured_body_part'] = "Torso"
df.loc[(df['WCIO Part Of Body Description'] == "HEART"), 'injured_body_part'] = "Torso"






c:\Users\govindd1\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\Users\govindd1\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [67]:
df['injured_body_part'].value_counts()

Arms           746108
Legs           614453
Spinal         382358
Face           217578
Multiple       208638
Others         137343
Torso          115338
Respiratory     31248
Name: injured_body_part, dtype: int64

In [68]:
df['Carrier Name'] = df['Carrier Name'].str.replace(r'[^\w\s]+', '')
remove_words = ['CO', 'COMPANY', 'pls', 'CORP', 'CORPORATION', 'DIST', 'DISTRICT', 'INC', 'SCH', 'CSD', 'OF', 'C S D', 'COR', 'INS', 'DT', 'CT', 'LLC']
pat = r'\b(?:{})\b'.format('|'.join(remove_words))
df['Carrier Name'] = df['Carrier Name'].str.replace(pat, '')

C:\Users\govindd1\AppData\Local\Temp/ipykernel_15768/2948151100.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Carrier Name'] = df['Carrier Name'].str.replace(r'[^\w\s]+', '')
C:\Users\govindd1\AppData\Local\Temp/ipykernel_15768/2948151100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Carrier Name'] = df['Carrier Name'].str.replace(r'[^\w\s]+', '')
C:\Users\govindd1\AppData\Local\Temp/ipykernel_15768/2948151100.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Carrier Name'] = df['Carrier Name'].str.replace(pat, '')
C:\Users\govindd1\AppData\Local\Temp/ipykernel_15768/2948151100.py:4: SettingWithCopyWarning: 
A value is try

In [ ]:
df.to_csv('./Cleaned_workers_compensation_data.csv')

### --------------------------------------  All the Cleaning Work Done in the Cells Above --------------------------------------

In [ ]:
df['Carrier Name'].value_counts()

In [ ]:
df['OIICS Part Of Body Description'].value_counts().plot(kind='barh', subplots=True, figsize=(20,10))

In [ ]:

age_distribution = df['Age at Injury'].value_counts().plot(kind = 'bar', subplots=True, figsize = (30,10))


In [ ]:
df['Current Claim Status'].value_counts().plot(kind='barh', subplots = True, figsize=(30,10))

In [ ]:
df['OIICS Nature of Injury Description'].value_counts().plot(kind='barh', subplots=True, figsize=(20,10))

In [ ]:
df['Carrier Name'] = df['Carrier Name'].str.replace(r'[^\w\s]+', '')
remove_words = ['CO', 'COMPANY', 'pls', 'CORP', 'CORPORATION', 'DIST', 'DISTRICT', 'INC', 'SCH', 'CSD', 'OF', 'C S D', 'COR', 'INS', 'DT', 'CT', 'LLC']
pat = r'\b(?:{})\b'.format('|'.join(remove_words))
df['Carrier Name'] = df['Carrier Name'].str.replace(pat, '')

# group by carrier name

In [ ]:
df['Carrier Name'].value_counts()

In [ ]:
group_by_carrier = df.groupby(by=['Carrier Name'])


In [ ]:


avg_days_to_settle_claim = group_by_carrier['days_btw_claim_est_and_negotiated'].mean()
avg_days_to_settle_claim


In [ ]:
group_by_carrier.mean()

In [ ]:
carrier_gpd = group_by_carrier.get_group('STATE INSURANCE FUND')

In [ ]:
carrier_gpd['days_btw_claim_est_and_negotiated'].mean()

# Grouping by Status of claims EDA



In [ ]:
group_by_label = df.groupby(by=['Current Claim Status'])
gpd_settled = group_by_label.get_group('NO FURTHER ACTION')

In [ ]:
gpd_settled

In [ ]:
gpd_settled['accident_year'].value_counts().plot(kind = 'bar', subplots=True, figsize = (30,10))

In [ ]:
gpd_settled['Age at Injury'].value_counts().plot(kind = 'bar', subplots=True, figsize = (30,10))

In [ ]:
gpd_settled['OIICS Nature of Injury Description'].value_counts().plot(kind='barh', subplots=True, figsize=(20,10))

In [ ]:
df.columns

In [ ]:
'OIICS Event Exposure Description', 'OIICS Injury Source Code',
       'OIICS Injury Source Description', 'OIICS Nature of Injury Code',
       'OIICS Nature of Injury Description', 'OIICS Part Of Body Code',
       'OIICS Part Of Body Description', 'OIICS Secondary Source Code',
       'OIICS Secondary Source Description', 'PPD Non-Scheduled Loss Date',

In [ ]:
df['OIICS Event Exposure Description'].value_counts().plot(kind='barh', subplots=True, figsize=(20,10))

In [ ]:
df['OIICS Injury Source Description'].value_counts().plot(kind='barh', subplots=True, figsize=(20,20))

In [ ]:
group_by_label = df.groupby(by=['accident_year'])
year_grpd_df = group_by_label.get_group(2018)

In [ ]:
year_grpd_df

In [ ]:
year_grpd_df['WCIO Cause of Injury Description'].value_counts().plot(kind='barh', subplots=True, figsize=(20,20))

In [ ]:
year_grpd_df['WCIO Cause of Injury Description'].value_counts()